In [1]:
import os
import pandas as pd

In [2]:
region_sz = ['罗湖区','福田区','南山区','盐田区','宝安区','龙华区','龙岗区','坪山区','光明区']

# Age

In [20]:
## 把罗湖与罗湖区的数据合在一起，然后删除罗湖区的数据
def operate_region(df):
    df_new = df.copy(deep=True)
    for i in region_sz:
        df_new["location_name"]=df_new["location_name"].str.replace(i,i[0:-1])
    return df_new

In [21]:
# stage_csv = pd.DataFrame()
def run_in_group(your_list,root_dir):
    for dir_name in your_list:
        file_list = os.listdir(os.path.join(root_dir, dir_name))
        stage_csv = pd.DataFrame()

        ## concat all csv files in one age group
        for file in file_list:
            one_csv = pd.read_csv(os.path.join(root_dir, dir_name, file), encoding='utf-16', sep='\t')
            stage_csv = pd.concat([stage_csv, one_csv], axis=0)

        ## "罗湖" 与 "罗湖区" 处理
        stage_csv2 = operate_region(stage_csv)

        final_csv = stage_csv2.groupby("location_name").size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
        final_csv_rmsz = final_csv.loc[final_csv["location_name"]!="深圳",:]

        final_csv_rmsz.to_csv(f"{root_dir}/{dir_name}.csv", index=False,sep='\t',encoding='utf-16')
    # break

In [22]:
def allcsv_to_onexlsx(xlsx_name,root_dir):
    # 创建一个ExcelWriter对象
    with pd.ExcelWriter(os.path.join(root_dir, xlsx_name), engine='xlsxwriter') as writer:
        tmp_list = os.listdir(root_dir)[0:-1]
        csv_list = [i for i in tmp_list if ".csv" in i]

        for file in csv_list:
            file_path = os.path.join(root_dir, file)
            one_csv = pd.read_csv(file_path, encoding='utf-16', sep='\t')
            # 将每个CSV文件写入Excel的不同工作表中
            sheet_name = f"{file.split('.')[0]}"
            one_csv.to_excel(writer, sheet_name=sheet_name, index=False)


In [17]:
age_dir = "data/地名实体识别汇总（round1-4） - 出生年份分开"
age_list = os.listdir(age_dir)
age_list = [i for i in age_list if "." not in i]
age_list

['1950-1960',
 '1960-1970',
 '1970-1980',
 '1980-1990',
 '1990-2000',
 '2000+',
 'less1950',
 'others']

In [ ]:
run_in_group(age_list,age_dir)
allcsv_to_onexlsx('出生年份频率汇总.xlsx',age_dir)

# gender

In [23]:
gender_dir = "data/地名实体识别汇总（round1-4） - 男女分开"
gender_list = os.listdir(gender_dir)
gender_list = [i for i in gender_list if "." not in i]
gender_list

['man', 'others', 'woman']

In [24]:
run_in_group(gender_list,gender_dir)
allcsv_to_onexlsx('性别频率汇总.xlsx',gender_dir)

# Come to sz

In [25]:
comesz_dir = "data/地名实体识别汇总（round1-4） - 抵深年份分开"
comesz_list = os.listdir(comesz_dir)
comesz_list = [i for i in comesz_list if "." not in i]
comesz_list

['1970-1980', '1980-1990', '1990-2000', '2000-2010', '2010+', 'others']

In [27]:
run_in_group(comesz_list,comesz_dir)
allcsv_to_onexlsx('来深频率汇总.xlsx',comesz_dir)